In [1]:
import requests
import json
import pandas as pd
import datetime as dt

In [2]:
s = requests.Session()

In [3]:
api_url = 'http://localhost:5000/api/v1/resources/{}'

In [24]:
r = s.get(api_url.format('get_chores'), params={'personId': 4}).json()

In [25]:
# r.content.decode() # 
r['results']

'{"ChoreID":{"0":1,"1":2,"2":3,"3":4,"4":5,"5":6,"6":7,"7":8,"8":9},"name":{"0":"Two week test","1":"One week test","2":"Today test","3":"Two day yesterday test","4":"Two day today test","5":"Two day today inactive test","6":"Test API chore","7":"Test Form chore","8":"Mr Michael E Roberts"},"schedule":{"0":"1W","1":"1W","2":"1W","3":"2D","4":"2D","5":"2D","6":"2D","7":"1D","8":"1W"},"start_date":{"0":"2023-01-05","1":"2023-01-12","2":"2023-01-19","3":"2023-01-18","4":"2023-01-19","5":"2023-01-19","6":"2023-01-28","7":"2023-01-28","8":"2023-01-28"},"start_time":{"0":"07:00:00","1":"07:00:00","2":"07:00:00","3":"07:00:00","4":"07:00:00","5":"07:00:00","6":"08:00:00","7":"(datetime.time(7, 0),)","8":"10"},"window":{"0":"4H","1":"4H","2":"4H","3":"4H","4":"4H","5":"4H","6":"6H","7":"4H","8":"12H"},"repeats":{"0":1,"1":1,"2":1,"3":1,"4":1,"5":1,"6":0,"7":1,"8":0},"active":{"0":1,"1":1,"2":1,"3":1,"4":1,"5":0,"6":1,"7":0,"8":1}}'

In [26]:
df = pd.DataFrame(json.loads(r['results']))
# list(zip(df['ChoreInstanceID'], df['name'], df['Completed']))
df

,ChoreID,name,schedule,start_date,start_time,window,repeats,active
0,1,Two week test,1W,2023-01-05,07:00:00,4H,1,1
1,2,One week test,1W,2023-01-12,07:00:00,4H,1,1
2,3,Today test,1W,2023-01-19,07:00:00,4H,1,1
3,4,Two day yesterday test,2D,2023-01-18,07:00:00,4H,1,1
4,5,Two day today test,2D,2023-01-19,07:00:00,4H,1,1
5,6,Two day today inactive test,2D,2023-01-19,07:00:00,4H,1,0
6,7,Test API chore,2D,2023-01-28,08:00:00,6H,0,1
7,8,Test Form chore,1D,2023-01-28,"(datetime.time(7, 0),)",4H,1,0
8,9,Mr Michael E Roberts,1W,2023-01-28,10,12H,0,1


In [104]:
for i in [9, 10,11,12]:
    s.post(api_url.format('uncomplete_chore_instance'), params={'choreInstanceId': i}) # .json()

In [88]:
r

<Response [200]>

In [7]:
#    name: str = None,
#    schedule: str = '1D',
#    start_date: dt.date = dt.date.today(),
#    start_time: dt.time = dt.time(7),
#    window: str = '4H',
#    repeats: bool = True,
#    active: bool = True,

s.post(api_url.format('create_chore'), 
    headers={
            "Content-type": "application/x-www-form-urlencoded; charset=UTF-8"
        },
    data={
            'name': 'Test API chore',
            'schedule': '2D',
            'start_date': dt.date.today() + dt.timedelta(days=1),
            'start_time': dt.time(8),
            'window': '6H',
            'repeats': False,
            'active': True
        }
    )

<Response [200]>